In [1]:
# L = instaloader.Instaloader( # Main Class info: https://instaloader.github.io/as-module.html#instaloader-main-class
#     download_pictures=False,
#     download_videos=False, 
#     download_video_thumbnails=False,
#     compress_json=False, 
#     download_geotags=False, 
#     post_metadata_txt_pattern="", 
#     max_connection_attempts=0,
#     download_comments=True,
#     save_metadata = False,
#     filename_pattern = '{mediaid}')
# post = instaloader.Post.from_mediaid(L.context, 2442527112545288493)
# L.update_comments('#bancocentral/2442527112545288493', post)
# L.download_post(post, target='#bancocentral')

# if not df_post.empty:
#     print(df_post['location_name']) 

In [26]:
pd.__version__

'1.0.5'

In [2]:
import datetime as dt
from itertools import dropwhile, takewhile
import instaloader
import pandas as pd
import re
import json
import os
import logging
import argparse
logging.basicConfig(level = logging.INFO)
logger = logging.getLogger('__instagram__')

In [3]:
def get_post_hashtags(since, until, hashtag):
    L = instaloader.Instaloader( # Main Class info: https://instaloader.github.io/as-module.html#instaloader-main-class
        download_pictures=False,
        download_videos=False, 
        download_video_thumbnails=False,
        compress_json=False, 
        download_geotags=False, 
        post_metadata_txt_pattern="", 
        max_connection_attempts=0,
        download_comments=True,
        save_metadata = False,
        filename_pattern = '{mediaid}'
    )
    posts = L.get_hashtag_posts(hashtag) # instaloader.Hashtag.get_posts().
    print("capturing posts from: "+str(since)+" to: "+str(until))
    SINCE = dt.datetime.strptime(until, "%Y-%m-%d %H:%M:%S") + dt.timedelta(hours=+5)
    UNTIL = dt.datetime.strptime(since, "%Y-%m-%d %H:%M:%S") + dt.timedelta(hours=+5)
    list_post = []
    for post in takewhile(lambda p: p.date >= UNTIL, dropwhile(lambda p: p.date >= SINCE, posts)):
        dict_post = {}
        dict_post['post_id'] = post.mediaid
        dict_post['date'] = post.date_local
        dict_post['user_id'] = post.owner_id
        dict_post['user_name'] = post.owner_username
        dict_post['text'] = post.caption
        dict_post['post_url'] = f'https://www.instagram.com/p/{post.shortcode}/'
        dict_post['post_type'] = post.typename
        dict_post['hashtags'] = post.caption_hashtags
        dict_post['mentions'] = post.tagged_users
        dict_post['replies'] = post.comments
        dict_post['likes'] = post.likes
        dict_post['url_picture'] = post.url
        dict_post['location_id'] =  post.__dict__['_full_metadata_dict']['location']['id'] if post.__dict__['_full_metadata_dict']['location'] else ''
        dict_post['location_name'] = post.__dict__['_full_metadata_dict']['location']['name'] if post.__dict__['_full_metadata_dict']['location'] else ''
#         dict_post['location'] = post.location
        dict_post['is_video'] = post.is_video
        dict_post['video_duration'] = post.video_duration
        dict_post['video_url'] = post.video_url
        dict_post['video_view'] = post.video_view_count
        list_post.append(dict_post)
        logger.info(f'{post.date_local} | {post.mediaid} | {post.caption}')
        L.download_post(post, target=f'#{hashtag}')
    return pd.DataFrame(list_post)

In [4]:
def get_fields_comment(comment, parent_id, root_id):
    try: 
        is_verified = comment['owner']['is_verified']
    except:
        is_verified = ''
    dict_comment = {'comment_id':comment['id'], 
                    'created_at':comment['created_at'],
                    'text':comment['text'],
                    'user_id':comment['owner']['id'],
                    'is_verified':is_verified,
                    'picture_url':comment['owner']['profile_pic_url'],
                    'user_name':comment['owner']['username'],
                    'likes_count':comment['likes_count'],
                    'parent_id':parent_id,
                    'root_id':root_id
                   }
    logger.info(f"{comment['id']} | {comment['text']} | {comment['owner']['username']}")
    return dict_comment

def get_list_comments(filename):
    root_id = re.sub(r'\D+', '', filename)
    with open(filename) as fp:
        data = json.load(fp)
    list_comment = []
    for comment in data:
        list_comment.append(get_fields_comment(comment, root_id, root_id))
        if comment['answers']:
            for replies in comment['answers']:
                list_comment.append(get_fields_comment(replies, comment['id'], root_id))
    return list_comment

In [5]:
def get_user_info(username):
    L = instaloader.Instaloader( # Main Class info: https://instaloader.github.io/as-module.html#instaloader-main-class
        download_pictures=False,
        download_videos=False, 
        download_video_thumbnails=False,
        compress_json=False, 
        download_geotags=False, 
        post_metadata_txt_pattern=None, 
        max_connection_attempts=0,
        download_comments=False,
    )
    dict_user_info = {}
    profile = instaloader.Profile.from_username(L.context, username)
    dict_user_info['user_id'] = profile.userid
    dict_user_info['user_name'] = profile.username
    dict_user_info['user_fullname'] = profile.full_name
    dict_user_info['profile_url'] = f'https://www.instagram.com/{profile.username}' 
    dict_user_info['picture_url'] = profile.profile_pic_url 
    dict_user_info['biography'] = profile.biography
    dict_user_info['followees_count'] = profile.followees
    dict_user_info['followers_count'] = profile.followers 
    dict_user_info['highlight_count'] = profile.__dict__['_node']['highlight_reel_count']
    dict_user_info['post_count'] = profile.mediacount 
    dict_user_info['igtv_count'] = profile.igtvcount 
    dict_user_info['is_verified'] = profile.is_verified 
    dict_user_info['is_private'] = profile.is_private
    dict_user_info['is_business_account'] = profile.is_business_account
    dict_user_info['business_category'] = profile.business_category_name
    dict_user_info['business_email'] = profile.__dict__['_node']['business_email']
    dict_user_info['category_enum'] = profile.__dict__['_node']['category_enum']
    dict_user_info['external_url'] = profile.external_url
    logger.info(f'{profile.username} | {profile.userid} | {profile.full_name}')
    return dict_user_info

In [12]:
def main(hashtag, since, until, download_user):
    ### setear el hashtags
#     hashtag = 'bancocentral'
    ### extraer post del hashtags
    logger.info('STARTING SCRAPPER POST OF INSTAGRAM')
#     df_posts = get_post_hashtags('2020-11-15 10:00:00', '2020-11-15 16:00:00', hashtag)
    df_posts = get_post_hashtags(since, until, hashtag)
    df_posts.to_csv('../results/df_post.csv', sep = '|', index = False, encoding='utf 8')
    ### extraer comentarios de post
    logger.info("NOW, SCRAPPER COMMENTS OF THE POSTS")
    list_comment = []
    for filename in os.listdir(f'#{hashtag}'):
        list_comment.extend(get_list_comments(f'#{hashtag}/{filename}'))
    df_comments = pd.DataFrame(list_comment)
    df_comments.to_csv('../results/df_comments.csv', sep = '|', index = False, encoding='utf 8')
    if download_user:
        ### extraer user_info de los tweets
        logger.info("LAST, EXTRACTION USER INFO")
        list_user_info = []
        for username in set(df_posts['user_name']):
            list_user_info.append(get_user_info(username))
        df_user_info = pd.DataFrame(list_user_info)
        df_user_info.to_csv('../results/df_user_info.csv', sep = '|', index = False, encoding='utf 8')

In [23]:
if __name__=='main':
    parser = argparse.ArgumentParser()
    parser.add_argument('-hashtags',
                        dest = 'hashtags',
                        help = 'write a hashtags whithout #',
                        type = str)
    parser.add_argument('-since',
                    dest = 'since',
                    help = 'define the start_date to scraper %Y-%m-%d %H:%M:%S',
                    type = str)
    parser.add_argument('-until',
                        dest = 'until',
                        help = 'define the last_date to scraper %Y-%m-%d %H:%M:%S',
                        type = str)
    parser.add_argument('--download_user',
                        dest = 'download_user',
                        help = 'set to download information of the user',
                        action = 'store_true')    

In [8]:
# from multiprocessing import Pool
# pool = Pool(3)
# list_user_info = []
# for dict_user_info in pool.map(get_user_info, df_post['username']):
#     list_user_info.append(dict_user_info)

In [29]:
df_posts_ = pd.read_csv('desktop/scrapper_instagram/results/df_post.csv', sep = '|', encoding='utf 8')
df_posts_

,post_id,date,user_id,user_name,text,post_url,post_type,hashtags,mentions,replies,likes,url_picture,location_id,location_name,is_video,video_duration,video_url,video_view
0,2443173757630986320,2020-11-15 14:37:09,32930010682,dataclave,ANÁLISIS\n.\nEl Palacio de Hacienda prevé logr...,https://www.instagram.com/p/CHn5ZaMJrhQ/,GraphImage,"['fmi', 'dólar', 'bancocentral', 'redrado']",[],0,2,https://z-p42-instagram.faqp1-1.fna.fbcdn.net/...,NaN,NaN,False,NaN,NaN,NaN
1,2443171866937667897,2020-11-15 14:33:24,1091882006,tynmagazine,Uno de los líderes de Banco Central de Inglate...,https://www.instagram.com/p/CHn495WDDE5/,GraphImage,"['crypto', 'cryptomonedas', 'criptomonedas', '...",[],0,9,https://z-p42-instagram.faqp1-1.fna.fbcdn.net/...,NaN,NaN,False,NaN,NaN,NaN
2,2443141543152151890,2020-11-15 13:33:09,177690654,edukador.bitcoin,#bitcoin #bancocentral #sistemadereservasfraci...,https://www.instagram.com/p/CHnyEoHlW1S/,GraphImage,"['bitcoin', 'bancocentral', 'sistemadereservas...",[],1,5,https://z-p42-instagram.faqp1-1.fna.fbcdn.net/...,NaN,NaN,False,NaN,NaN,NaN
3,2443111873684771131,2020-11-15 12:34:12,6448050331,investidorsa,"Quem fez isso, merecia ir direto para a prova ...",https://www.instagram.com/p/CHnrU4RhRE7/,GraphImage,[],[],27,1089,https://z-p42-instagram.faqp1-1.fna.fbcdn.net/...,2.130887e+08,"Brasília, Brazil",False,NaN,NaN,NaN
4,2443109978094265254,2020-11-15 12:30:26,3969004504,keniamota4589,Faça seu empréstimo\nAprovação imediata\n#empr...,https://www.instagram.com/p/CHnq5S3hWOm/,GraphImage,"['empreendedorismo', 'empreatimoconsignado', '...",[],0,1,https://z-p42-instagram.faqp1-1.fna.fbcdn.net/...,NaN,NaN,False,NaN,NaN,NaN
5,2443059940727162018,2020-11-15 10:51:01,22137118200,direitosimplesassimadv,"""Coluna da Sol"" - Solange Ribeiro (@sol.ribeir...",https://www.instagram.com/p/CHnfhJ8B4Si/,GraphSidecar,"['bancocentral', 'bacen', 'versos', 'informação']",[],0,9,https://z-p42-instagram.faqp1-1.fna.fbcdn.net/...,NaN,NaN,False,NaN,NaN,NaN
6,2443054741033090160,2020-11-15 10:40:41,1586640828,dipeizini,🎯O que é Taxa Selic?\nA Taxa Selic (Sistema ...,https://www.instagram.com/p/CHneVfWHHxw/,GraphImage,"['taxaselic', 'selic', 'copom', 'taxadejuros',...",['dipeizini'],0,9,https://z-p42-instagram.faqp1-1.fna.fbcdn.net/...,1.649807e+14,PEDRA AZUL - tudo de bom,False,NaN,NaN,NaN
7,2443049851155374058,2020-11-15 10:30:58,4902011251,contabilidadeimpuestos,⭐⭐⭐⭐⭐\n.\n.\n.\n\n.\n.\n.\n.\n#2020 #constituc...,https://www.instagram.com/p/CHndOVSr9_q/,GraphImage,"['2020', 'constituciondeempresas', 'contabilid...",[],0,3,https://z-p42-instagram.faqp1-1.fna.fbcdn.net/...,3.305078e+08,Distrito Nacional,False,NaN,NaN,NaN
8,2443039936372623018,2020-11-15 10:11:16,6168144327,jacksonpaivaoficial,"A partir de amanhã, dia 16, começa a operar of...",https://www.instagram.com/p/CHna-DblUaq/,GraphImage,"['pix', 'bancocentral', 'pixbancocentraldobras...",[],0,3,https://z-p42-instagram.faqp1-1.fna.fbcdn.net/...,NaN,NaN,False,NaN,NaN,NaN
9,2443038540289020694,2020-11-15 10:08:30,31300109960,estadaoinvestidor,📲 O PIX vai chegar no dia 16 de novembro e mui...,https://www.instagram.com/p/CHnapvOhBMW/,GraphImage,"['pix', 'bancocentral', 'einvestidor', 'infogr...",[],1,50,https://z-p42-instagram.faqp1-1.fna.fbcdn.net/...,NaN,NaN,False,NaN,NaN,NaN


In [30]:
df_posts_ = pd.read_csv('desktop/scrapper_instagram/results/df_comments.csv', sep = '|', encoding='utf 8')
df_posts_

,comment_id,created_at,text,user_id,is_verified,picture_url,user_name,likes_count,parent_id,root_id
0,17914695700495803,1605440882,👏👏,4224884027,False,https://z-p42-instagram.faqp1-1.fna.fbcdn.net/...,danielle_barbieri,0,2442931503581585559,2442931503581585559
1,17870117561088443,1605440156,#PoderTOPVIEW #Economia #pix #bancocentral #pa...,597475489,False,https://z-p42-instagram.faqp1-1.fna.fbcdn.net/...,topviewclub,0,2442931503581585559,2442931503581585559
2,17871638288023928,1605456911,Soon Moon !!!! go #bitcoin,44013767566,False,https://z-p42-instagram.faqp1-1.fna.fbcdn.net/...,ninacryptoshow,0,2442974402369085177,2442974402369085177
3,17913687670496443,1605484678,Si usted tiene un buen crédito y va a un banco...,3930060554,False,https://z-p42-instagram.faqp1-1.fna.fbcdn.net/...,josejoaquingomezperez,0,2443002606245066466,2443002606245066466
4,17862784592164376,1605473111,Si quemaran el partidos del PLD el país se arecla,4123389267,False,https://z-p42-instagram.faqp1-1.fna.fbcdn.net/...,fracasoantonio,0,2443002606245066466,2443002606245066466
...,...,...,...,...,...,...,...,...,...,...
190,17855891087304115,1605480294,"@maria_duda_na_lua certeza que é , eu já vir e...",1410058075,False,https://z-p42-instagram.faqp1-1.fna.fbcdn.net/...,reinan_lincoln,1,18135410158127771,2443111873684771131
191,17865701045112594,1605461740,rapaz 😂😂 pior que ficou bonita!!!,1357756142,False,https://z-p42-instagram.faqp1-1.fna.fbcdn.net/...,heniogabriell,5,2443111873684771131,2443111873684771131
192,17861853065225395,1605461802,@heniogabriell ficou mesmo. Quem fez isso é um...,6448050331,False,https://z-p42-instagram.faqp1-1.fna.fbcdn.net/...,investidorsa,3,17865701045112594,2443111873684771131
193,18122897260146903,1605461721,#investir #dinheiro #finanças #investimentosin...,6448050331,False,https://z-p42-instagram.faqp1-1.fna.fbcdn.net/...,investidorsa,0,2443111873684771131,2443111873684771131
